# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
#載入套件
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [2]:
# 查看目前上映那些電影，並擷取出其ID資訊
# 想要爬資料的目標網址
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url) #用 requests 的 get 方法把網頁抓下來
resp.encoding = 'utf-8'#轉換編碼至UTF-8

soup = BeautifulSoup(resp.text, 'lxml')# text 屬性就是 html 檔案，指定 lxml 作為解析器(lxml效率比較好)
html = soup.find("select", attrs={'name':'movie_id'}) # 找出一個"div",尋找正在上映中的全部電影筆數，其所在的tag
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 海上鋼琴師4K數位修復, ID: 10425
Movie: 鹿皮奇談, ID: 10420
Movie: 大約在冬季, ID: 10408
Movie: 倒數計死, ID: 10390
Movie: 吹哨人, ID: 10381
Movie: 幸福試用期, ID: 10374
Movie: 竊竊屍語, ID: 10365
Movie: 舞力決戰, ID: 10344
Movie: 潛艦追緝, ID: 10333
Movie: 霹靂嬌娃, ID: 10124
Movie: 白頭山：半島浩劫, ID: 10354
Movie: CATS貓, ID: 10166
Movie: 金魚, ID: 10424
Movie: 海上花, ID: 10376
Movie: 奪魂露營車, ID: 10366
Movie: 怨靈古堡, ID: 10356
Movie: 添好孕, ID: 10332
Movie: 外出偷馬, ID: 10310
Movie: 茱蒂, ID: 10112
Movie: 燃燒女子的畫像, ID: 10069
Movie: 葉問4：完結篇, ID: 9890
Movie: STAR WARS : 天行者的崛起, ID: 10225
Movie: 復仇母親, ID: 10392
Movie: 風暴過後, ID: 10387
Movie: 南方車站的聚會, ID: 10380
Movie: 祭旗, ID: 10373
Movie: 黑熊來了, ID: 10357
Movie: 黑計畫, ID: 10351
Movie: HUMAN LOST人間失格, ID: 10348
Movie: 假面騎士劇場版ZI-O Over Quartzer, ID: 10331
Movie: 每個月來訪一次的月經醬, ID: 10324
Movie: 8級警戒, ID: 10323
Movie: 為你存在的每一天, ID: 10297
Movie: 黑色聖誕節, ID: 10291
Movie: 熱氣球飛行家, ID: 10257
Movie: 大說謊家, ID: 10122
Movie: 特約經紀公司, ID: 10364
Movie: 花生醬獵鷹的願望, ID: 10340
Movie: 人間失格：太宰治與他的3個女人, ID: 10287
Movie: 呆萌特務, I

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [5]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 9597

In [6]:
# 想要爬資料的目標網址
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 彰化, 代號(area_id): 22
放映地區: 南投, 代號(area_id): 13
放映地區: 雲林, 代號(area_id): 19
放映地區: 嘉義, 代號(area_id): 21
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 屏東, 代號(area_id): 14
放映地區: 基隆, 代號(area_id): 18
放映地區: 宜蘭, 代號(area_id): 11
放映地區: 花蓮, 代號(area_id): 12
放映地區: 台東, 代號(area_id): 9
放映地區: 澎湖, 代號(area_id): 23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [6]:
# 指定放映地區
area_id = 28

In [8]:
# 向網站發送請求
# 想要爬資料的目標網址
url = 'https://movies.yahoo.com.tw/movietime_result.html'
# 將查詢參數加入 GET 請求中
payload = {'movie_id':str(movie_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8' #轉換編碼至UTF-8

# 取回請求的回覆
soup = BeautifulSoup(resp.text, 'lxml') # text 屬性就是 html 檔案，指定 lxml 作為解析器(lxml效率比較好)
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})# 找出所有"label",尋找正在上映中的全部電影筆數，其所在的tag

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

十二月 31
一月 1
一月 2
一月 3
一月 4


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [9]:
# 選定要觀看的日期
date = "2019-08-21"

In [12]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [10]:
for lists in html:
#     print(type(lists))
#     print(lists.a.text)
    print('===============================================================================================================')
    print("電影院名稱:{0}".format(lists.a.text,))
    mTypes = lists.find_all('input',{'class':'gabtn'})
    for mType in mTypes:
        print("影片名稱:{0} 影片放映類型:{1} 時間表:{2}".format(mType['data-movie_title'],mType['data-movie_type'],mType['value']))
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''


'\n\n  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表\n  \n  Your code here.\n\n'